# Análise Exploratória de Dados - Gelateria Lillo

## Objetivo
Carregar os dados, identificar problemas (nulos, outliers) e preparar o terreno para a limpeza.

In [4]:
import pandas as pd
import os
from datetime import datetime
import ast
import matplotlib.pyplot as plt
import seaborn as sns

ModuleNotFoundError: No module named 'pandas'

## Carregamento dos Dados

In [ ]:
# Definir caminhos (ajuste conforme necessário)
portfolio_path = 'data/portfolio_ofertas.csv'
events_path = 'data/eventos_ofertas.csv'
profile_path = 'data/dados_clientes.csv'

# Carregar com encoding latin-1 para evitar erros
try:
    portfolio = pd.read_csv(portfolio_path, encoding='latin-1')
    events = pd.read_csv(events_path, encoding='latin-1')
    profile = pd.read_csv(profile_path, encoding='latin-1')
    print("Dados carregados com sucesso!")
except Exception as e:
    print(f"Erro ao carregar dados: {e}")

## Inspeção Inicial (Info e Describe)

In [ ]:
print("--- Portfolio ---")
display(portfolio.info())
display(portfolio.describe())

print("\n--- Eventos ---")
display(events.info())
display(events.describe())

print("\n--- Clientes ---")
display(profile.info())
display(profile.describe())

## Análise de Nulos (Renda e Gênero)

In [ ]:
null_metrics = profile[['renda_anual', 'genero']].isnull().mean() * 100
print("Percentual de nulos:")
print(null_metrics)

## Verificação de Idades Inconsistentes (> 100 anos)

In [ ]:
inconsistent_ages = profile[profile['idade'] > 100]
print(f"Registros com idade > 100: {len(inconsistent_ages)}")
if not inconsistent_ages.empty:
    display(inconsistent_ages[['idade']].head())

# Passo 2: Limpeza e Tipagem (Data Cleaning)

## Conversão de Tipos e Tratamento de Nulos

In [ ]:
# 1. Converter 'membro_desde' para datetime
# Formato esperado YYYYMMDD
profile['membro_desde'] = pd.to_datetime(profile['membro_desde'], format='%Y%m%d')
print("Conversão de 'membro_desde' concluída.")

# 2. Tratar Nulos em 'renda_anual' (Preencher com Mediana)
median_income = profile['renda_anual'].median()
profile['renda_anual'] = profile['renda_anual'].fillna(median_income)
print(f"Renda anual nula preenchida com mediana: {median_income}")

# 3. Tratar Nulos em 'genero' (Preencher com 'O' - Outros/Não Informado)
profile['genero'] = profile['genero'].fillna('O')
print("Gênero nulo preenchido com 'O'.")

# 4. Criar coluna 'anos_de_membro'
current_date = datetime.now()
profile['anos_de_membro'] = (current_date - profile['membro_desde']).dt.days / 365.25
print("Coluna 'anos_de_membro' criada.")

## Verificação Final do Passo 2

In [ ]:
display(profile.info())
display(profile[['membro_desde', 'renda_anual', 'genero', 'anos_de_membro']].head())

# Passo 3: Enriquecimento de Dados (The Big Merge)

## Unificação das Bases (Eventos + Portfolio + Clientes)

In [ ]:
# 1. Renomear colunas para chaves consistentes
portfolio = portfolio.rename(columns={'id': 'id_oferta'})
profile = profile.rename(columns={'id': 'cliente'})
print("Colunas renomeadas: portfolio['id'] -> 'id_oferta', profile['id'] -> 'cliente'.")

# 2. Merge Eventos + Portfolio (Left Join)
merged_df = events.merge(portfolio, on='id_oferta', how='left')

# 3. Merge Resultado + Clientes (Left Join)
final_df = merged_df.merge(profile, on='cliente', how='left')

print("Merges concluídos.")

## Validação do Merge

In [ ]:
# Verificar se número de linhas se manteve
initial_rows = len(events)
final_rows = len(final_df)

print(f"Linhas Iniciais (Eventos): {initial_rows}")
print(f"Linhas Finais (Merge): {final_rows}")

assert initial_rows == final_rows, "ERRO: Número de linhas foi alterado!"
print("SUCESSO: Contagem de linhas mantida.")

# Visualizar Schema Final
display(final_df.info())
display(final_df.head())

# Passo 4: Análise de Negócio e KPIs

## 1. Ticket Médio

In [ ]:
transactions = final_df[final_df['tipo_evento'] == 'transacao']
ticket_average = transactions['valor'].mean()
print(f"Ticket Médio: R$ {ticket_average:.2f}")

## 2. Taxa de Conversão por Canal

In [ ]:
# Explodir canais (estão como string de lista)
portfolio['canais_lista'] = portfolio['canal'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else [])
offer_channels = portfolio[['id_oferta', 'canais_lista']].explode('canais_lista')
offer_channels = offer_channels.rename(columns={'canais_lista': 'canal_individual'})

# Filtrar eventos relevantes
relevant_events = final_df[final_df['tipo_evento'].isin(['oferta visualizada', 'oferta concluída'])]
relevant_events = relevant_events[['id_oferta', 'tipo_evento']]

# Join para associar eventos aos canais da oferta
events_with_channels = relevant_events.merge(offer_channels, on='id_oferta', how='inner')

# Calcular conversão
channel_stats = events_with_channels.groupby(['canal_individual', 'tipo_evento']).size().unstack(fill_value=0)
if 'oferta concluída' in channel_stats.columns and 'oferta visualizada' in channel_stats.columns:
    channel_stats['taxa_conversao'] = channel_stats['oferta concluída'] / channel_stats['oferta visualizada']
    display(channel_stats.sort_values('taxa_conversao', ascending=False))
else:
    print("Colunas de conversao ausentes nos dados (verifique encoding/nomes).")
    display(channel_stats)

## 3. Receita por Tipo de Oferta

In [ ]:
# Atribuição de Receita: Cruzar 'oferta concluída' com 'transacao' por cliente e tempo
completed = final_df[final_df['tipo_evento'] == 'oferta concluída'][['cliente', 'tempo_decorrido', 'oferta']]
trans = final_df[final_df['tipo_evento'] == 'transacao'][['cliente', 'tempo_decorrido', 'valor']]

attributed_revenue = completed.merge(trans, on=['cliente', 'tempo_decorrido'], how='inner')

revenue_by_type = attributed_revenue.groupby('oferta')['valor'].sum().sort_values(ascending=False)
print("Receita Total por Tipo de Oferta:")
display(revenue_by_type)

# Passo 5: Segmentação de Clientes (RFM)

## 1. Calcular Recência, Frequência e Valor

In [ ]:
transactions = final_df[final_df['tipo_evento'] == 'transacao']
max_time = transactions['tempo_decorrido'].max()

rfm = transactions.groupby('cliente').agg({
    'tempo_decorrido': lambda x: max_time - x.max(), # Recência (Quanto menor, melhor - mas aqui estamos usando tempo decorrido, entao x.max() é o mais recente. Subtraindo do max total dá "horas atrás")
    'cliente': 'count', # Frequência
    'valor': 'sum' # Valor Monetário
})

rfm.rename(columns={
    'tempo_decorrido': 'Recency',
    'cliente': 'Frequency',
    'valor': 'Monetary'
}, inplace=True)

display(rfm.head())

## 2. Atribuir Scores (Quartis)

In [ ]:
# Recência: Menor é melhor (5)
# Usando qcut para dividir em 5 partes, labels invertidos
rfm['R_Score'] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1])

# Frequência: Maior é melhor (usando rank para lidar com empates)
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1, 2, 3, 4, 5])

# Monetário: Maior é melhor
rfm['M_Score'] = pd.qcut(rfm['Monetary'].rank(method='first'), 5, labels=[1, 2, 3, 4, 5])

display(rfm[['R_Score', 'F_Score', 'M_Score']].head())

## 3. Criar Segmentos

In [ ]:
def segment_customer(row):
    r = row['R_Score']
    fm = (row['F_Score'] + row['M_Score']) / 2
    
    if r >= 4 and fm >= 4:
        return 'Campeoes'
    elif r >= 3 and fm >= 3:
        return 'Clientes Leais'
    elif r <= 2 and fm >= 3:
        return 'Em Risco'
    elif r <= 2 and fm < 3:
        return 'Hibernando'
    elif r >= 3 and fm < 3:
        return 'Promissores'
    else:
        return 'Precisa de Atencao'

rfm['Segment'] = rfm.apply(segment_customer, axis=1)

print("Distribuição dos Segmentos:")
display(rfm['Segment'].value_counts())

# Passo 6: Visualização e Insights

## 1. Demografia dos Clientes

In [ ]:
# Configurar estilo
sns.set(style="whitegrid")

fig, ax = plt.subplots(1, 2, figsize=(14, 5))

# Histograma de Idade
sns.histplot(final_df['idade'].drop_duplicates(), bins=30, kde=True, ax=ax[0])
ax[0].set_title('Distribuição de Idade')

# Countplot de Gênero
sns.countplot(data=final_df[['cliente', 'genero']].drop_duplicates(), x='genero', ax=ax[1])
ax[1].set_title('Distribuição de Gênero')

plt.show()

## 2. Funil de Ofertas (Visualizada vs Concluída)

In [ ]:
# Filtrar eventos
funnel_data = final_df[final_df['tipo_evento'].isin(['oferta visualizada', 'oferta concluída'])]
# Agrupar por Oferta e Tipo de Evento
funnel_counts = funnel_data.groupby(['oferta', 'tipo_evento']).size().reset_index(name='contagem')

plt.figure(figsize=(12, 6))
sns.barplot(data=funnel_counts, x='oferta', y='contagem', hue='tipo_evento')
plt.title('Funil de Oferta: Visualizada vs Concluída')
plt.ylabel('Quantidade de Eventos')
plt.show()

## 3. Clusters RFM (Recência vs Monetário)

In [ ]:
plt.figure(figsize=(10, 6))
# Scatter plot filtrando apenas segmentos principais para não poluir, ou todos
sns.scatterplot(data=rfm, x='Recency', y='Monetary', hue='Segment', alpha=0.6)
plt.title('RFM Clusters: Recência vs Valor Monetário')
plt.xlabel('Recência (Horas desde última compra)')
plt.ylabel('Valor Total Gasto ($)')
plt.yscale('log') # Escala log
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

# Conclusão

O projeto seguiu todas as etapas propostas: carregamento, limpeza, merge, KPIs, Segmentação e Visualização. A base está pronta para análises mais profundas ou integração com modelos preditivos.